In [20]:
import pandas as pd
from datetime import datetime
from scipy.stats import norm
from math import sqrt
import numpy as np


# задание 1

### найти мин размер группы

In [5]:
data = pd.read_csv('2022-04-01T12_df_sales.csv')
data['date'] = data.apply(lambda x: pd.to_datetime(x['date']), axis=1)
data_slice = data[(data['date'] >= datetime(2022, 2, 21)) & (data['date'] < datetime(2022, 2, 28))].copy(deep=True)


In [6]:
data_per_user = data_slice[['user_id', 'price']].groupby('user_id').sum('price').reset_index()

In [7]:
data_per_user.head(3)

,user_id,price
0,00045f,720
1,0006bb,1260
2,000b52,3480


In [13]:
def calculate_size(df, alpha=0.05, betta=0.1, effect_size=20):
    std_dev = df['price'].std()
    n_min = ((norm.ppf(1 - betta, loc=0, scale=1) + norm.ppf(1 - alpha/2, loc=0, scale=1))**2)
    n_min *= (2 * (std_dev**2))
    n_min /= effect_size**2
    
    return n_min
    

In [14]:
n_min = calculate_size(data_per_user)

In [15]:
print(n_min // 10 * 10)

34570.0


# задание 2

### найти мин размер эффекта

In [16]:
def calculate_effect(df, alpha=0.05, betta=0.1):
    std_dev = df['price'].std()
    n_min = ((norm.ppf(1 - betta, loc=0, scale=1) + norm.ppf(1 - alpha/2, loc=0, scale=1))**2)
    n_min *= (2 * (std_dev**2))
    n_min /= (len(df) / 2)
    n_min = sqrt(n_min)
    
    return n_min
    

In [17]:
min_effect = calculate_effect(data_per_user)
print(min_effect)


33.36719696756425


# задание 3

### реализация estimate_sample_size

In [72]:
import pandas as pd
from datetime import datetime
from scipy.stats import norm
from math import sqrt
import numpy as np


def estimate_sample_size(metrics, effect, alpha, beta):
    """
    Оцениваем необходимый размер выборки для проверки гипотезы о равенстве средних.
    
    Для метрик, у которых для одного пользователя одно значение просто вычислите
    размер групп по формуле.
    
    Для метрик, у которых для одного пользователя несколько значений (например,
    response_time), вычислите необходимый объём данных и разделите его на среднее
    количество значений на одного пользователя.
    
    Пример, если в таблице metrics 1000 наблюдений и 100 уникальных пользователей,
    и для эксперимента нужно 302 наблюдения, то размер групп будет 31, тк в среднем на
    одного пользователя 10 наблюдений, то получится порядка 310 наблюдений в группе.

    :param metrics (pd.DataFrame): таблица со значениями метрик,
        содержит столбцы ['user_id', 'metric'].
    :param effect (float): размер эффекта в процентах.
        Пример, effect=3 означает, что ожидаем увеличение среднего на 3%.
    :param alpha (float): уровень значимости.
    :param beta (float): допустимая вероятность ошибки II рода.
    :return (int): минимально необходимый размер групп (количество пользователей)
    """
    if len(metrics) == metrics.user_id.nunique():
        
        std_dev = np.std(metrics['metric'])
        lin_effect = metrics['metric'].mean() * effect / 100
        
        min_sample_size = ((norm.ppf(1 - beta, loc=0, scale=1) + norm.ppf(1 - alpha/2, loc=0, scale=1))**2)
        min_sample_size *= (2 * (std_dev**2))
        min_sample_size /= (lin_effect**2)
        
    else:
        
        std_dev = np.std(metrics['metric'])
        lin_effect = metrics['metric'].mean() * effect / 100
        
        min_sample_size = ((norm.ppf(1 - beta, loc=0, scale=1) + norm.ppf(1 - alpha/2, loc=0, scale=1))**2)
        min_sample_size *= (2 * (std_dev**2))
        min_sample_size /= (lin_effect**2)
        
        samples_per_user = len(metrics) / metrics.user_id.nunique()
        
        min_sample_size /= samples_per_user
        
        
    return min_sample_size
    

In [73]:
metrics = pd.DataFrame({
    'user_id': np.arange(100),
    'metric': np.linspace(500, 1490, 100)
})
effect, alpha, beta = 3, 0.05, 0.1
sample_size = estimate_sample_size(metrics, effect, alpha, beta)
# sample_size = 1966


In [75]:
sample_size

1965.2276493456436

In [76]:
metrics = pd.DataFrame({
    'user_id': np.arange(100) % 30,
    'metric': np.linspace(500, 1490, 100)
})
effect, alpha, beta = 3, 0.05, 0.1
sample_size = estimate_sample_size(metrics, effect, alpha, beta)
# sample_size = 590

In [77]:
sample_size

589.5682948036931